# 🎵 Spotify Data Analysis & Machine Learning Project

**Öğrenci:** Firuze Eroğlu (201613709044)  
**Ders:** Machine Learning (Fall 2025)

Bu proje, Spotify verilerini kullanarak uçtan uca üç farklı makine öğrenmesi problemini ele almaktadır:
1.  **Sınıflandırma (Classification):** Şarkı özelliklerine göre popülerlik tahmini.
2.  **Regresyon (Regression):** Şarkıların popülerlik puanının (0-100) tahmini.
3.  **Kümeleme (Clustering):** Şarkıların dinlenme sayılarına göre segmentasyonu.

---

## 1. Kütüphanelerin Yüklenmesi ve Ayarlar
Gerekli olan `pandas`, `sklearn`, `xgboost` gibi veri bilimi kütüphaneleri içe aktarılır.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import time

# Scikit-Learn Modülleri
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, _classification_report, r2_score, mean_squared_error)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import xgboost as xgb
import shap

# Uyarıları kapat (Temiz çıktı için)
warnings.filterwarnings('ignore')

## 2. Veri Setlerinin Yüklenmesi (Otomatik)
Bu kod bloğu, çalışma ortamını (Local veya Google Colab) algılar. Eğer veri setleri yerel dizinde yoksa, **GitHub deposundan otomatik olarak indirir.**

In [ ]:
# --- VERİ İNDİRME FONKSİYONU ---
def load_data_from_github(filename, url):
    # 1. Dosya yerelde var mı kontrol et
    if not os.path.exists(filename):
        print(f"[BILGI] '{filename}' bulunamadı, GitHub'dan indiriliyor...")
        try:
            # Wget komutu ile dosyayı çek
            os.system(f'wget -O "{filename}" "{url}"')
            print("[BASARILI] İndirme tamamlandı.")
        except Exception as e:
            print(f"[HATA] İndirme başarısız: {e}")
    else:
        print(f"[BILGI] '{filename}' zaten mevcut, yükleniyor...")
    
    # 2. Dosyayı Oku (Encoding hatasına karşı önlem alarak)
    try:
        df = pd.read_csv(filename)
    except:
        df = pd.read_csv(filename, encoding='ISO-8859-1')
    return df

# URL Tanımları
URL_CLASS = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/dataset.csv"
URL_REG = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/spotify_data%20clean.csv"
URL_CLUST = "https://raw.githubusercontent.com/frzerxz/spotify-ml-analysis/main/Most_Streamed_Spotify_Songs_2024.csv"



## 3. Sınıflandırma Görevi (Classification)
**Amaç:** Şarkının ses özelliklerine (dans edilebilirlik, enerji vb.) bakarak 'Popüler' olup olmadığını tahmin etmek.

In [ ]:
# 1. Veriyi Yükle
df_class = load_data_from_github('dataset.csv', URL_CLASS)
print(f"Veri Seti Boyutu: {df_class.shape}")

# 2. Hedef Değişkenin Oluşturulması
# Popülaritesi 50'den büyük olanlar '1' (Popüler), diğerleri '0' (Popüler Değil)
df_class['is_popular'] = (df_class['popularity'] > 50).astype(int)

# 3. Gereksiz Sütunların Temizlenmesi
# ID, İsim gibi makine öğrenmesine katkısı olmayan sütunlar çıkarılır.
cols_to_drop = ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'track_genre']
X = df_class.drop(columns=[c for c in cols_to_drop if c in df_class.columns])
X = X.drop(columns=['is_popular']) # Hedef değişkeni özelliklerden ayır
y = df_class['is_popular']

# Explicit sütununu sayısal hale getir
if 'explicit' in X.columns:
    X['explicit'] = X['explicit'].astype(int)

print("Öznitelik Seçimi Tamamlandı.")



### Veri Ön İşleme ve Dengesizlik Giderme
Eksik veriler doldurulur, veriler ölçeklenir ve `RandomUnderSampler` ile sınıf dengesizliği giderilir.

In [ ]:
# Eğitim ve Test Ayrımı (%80 Eğitim, %20 Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Pipeline Tanımlama
# Sayısal veriler için: Eksik verileri doldur -> Standartlaştır (Scale)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numeric_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)
    ])

# Dengesiz Veri Yönetimi (Undersampling)
# Popüler olmayan şarkı sayısı çok fazla olduğu için, popüler olanlarla eşitliyoruz.
try:
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler(random_state=42)
    # Pipeline uygulamadan önce veriyi dengelemek en sağlıklısıdır
    print(f"Resampling Öncesi Sınıf Dağılımı: {np.bincount(y_train)}")
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    print(f"Resampling Sonrası Sınıf Dağılımı: {np.bincount(y_train_resampled)}")
    
    X_train = X_train_resampled
    y_train = y_train_resampled
except ImportError:
    print("[UYARI] imblearn kütüphanesi bulunamadı, orjinal veri ile devam ediliyor.")

# Veriyi Dönüştür (Fit & Transform)
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
print("Ön işleme tamamlandı.")



### Model Eğitimi ve Değerlendirme
Random Forest modeli eğitilir ve sonuçlar raporlanır.

In [ ]:
# Model Eğitimi: Random Forest Classifier
print("Random Forest modeli eğitiliyor...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_processed, y_train)

# Tahmin
y_pred = rf_model.predict(X_test_processed)

# Sonuçlar
print("--- Sınıflandırma Sonuçları ---")
print(f"Doğruluk (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
# Detaylı Rapor (Precision, Recall, F1)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



## 4. Regresyon Görevi (Regression)
**Amaç:** Şarkının ve sanatçının metadatalarını kullanarak 0-100 arasındaki 'Popülerlik Puanını' net olarak tahmin etmek.

In [ ]:
# 1. Veriyi Yükle
df_reg = load_data_from_github('spotify_data clean.csv', URL_REG)
print(f"Regresyon Veri Seti Boyutu: {df_reg.shape}")

# 2. Özellik Seçimi
target_col = 'track_popularity'
# Metin tabanlı ve gereksiz sütunları çıkarıyoruz
cols_to_drop = ['track_id', 'track_name', 'artist_name', 'album_id', 'album_name', 'album_release_date', 'artist_genres']
X_reg = df_reg.drop(columns=[c for c in cols_to_drop if c in df_reg.columns])
X_reg = X_reg.drop(columns=[target_col])
y_reg = df_reg[target_col]

# 3. Pipeline Hazırlığı
numeric_features_reg = X_reg.select_dtypes(include=['int64', 'float64']).columns
categorical_features_reg = X_reg.select_dtypes(include=['object', 'bool']).columns

reg_preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numeric_features_reg),
    ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features_reg)
])

# Eğitim/Test Ayrımı
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)



In [ ]:
# Model Eğitimi: Random Forest Regressor
print("Regresyon Modeli Eğitiliyor (Bu işlem birkaç saniye sürebilir)...")
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Pipeline Oluştur (Ön İşleme + Model)
reg_pipeline = Pipeline(steps=[('preprocessor', reg_preprocessor), ('regressor', rf_reg)])
reg_pipeline.fit(X_train_reg, y_train_reg)

# Tahmin ve Değerlendirme
y_pred_reg = reg_pipeline.predict(X_test_reg)

r2 = r2_score(y_test_reg, y_pred_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))

print(f"--- Regresyon Sonuçları ---")
print(f"R2 Skoru (Başarı Oranı): {r2:.4f}")
print(f"RMSE (Ortalama Hata Payı): {rmse:.2f}")



## 5. Kümeleme Görevi (Clustering)
**Amaç:** Şarkıları dinlenme sayılarına (Streams) ve sosyal medya etkileşimlerine (TikTok/YouTube) göre gruplara ayırmak.
Bu veri seti etiketli değildir (Unsupervised Learning).

In [ ]:
# 1. Veriyi Yükle
filename_clust = 'Most_Streamed_Spotify_Songs_2024.csv'
df_clust = load_data_from_github(filename_clust, URL_CLUST)

# 2. Veri Temizliği
# Sayısal olması gereken sütunlardaki virgülleri (örn: "1,000") temizliyoruz.
cols_to_clean = ['Spotify Streams', 'Spotify Playlist Count', 'YouTube Views', 'TikTok Views']
for col in cols_to_clean:
    if col in df_clust.columns and df_clust[col].dtype == object:
        df_clust[col] = pd.to_numeric(df_clust[col].astype(str).str.replace(',', ''), errors='coerce')

# Sadece sayısal sütunları al ve eksikleri 0 ile doldur
X_clust = df_clust[cols_to_clean].fillna(0)

# Veri çok büyükse hız için örneklem al
if len(X_clust) > 5000:
    X_clust = X_clust.iloc[:5000]

# 3. Ölçekleme (Clustering için kritiktir)
scaler_clust = StandardScaler()
X_clust_scaled = scaler_clust.fit_transform(X_clust)

# 4. K-Means Modelleme
print("K-Means Kümeleme Çalıştırılıyor (K=3)...")
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X_clust_scaled)

# 5. Başarı Metriği (Silhouette Score)
sil_score = silhouette_score(X_clust_scaled, labels)
print(f"--- Kümeleme Sonucu ---")
print(f"Silhouette Skoru: {sil_score:.4f} (1'e ne kadar yakınsa o kadar iyi ayrışmış demektir)")

